In [1]:
import pandas as pd

In [73]:
mp_stat = pd.read_csv('mpids_stat_all.csv', index_col=0)
mp_stat['elements'] = mp_stat['elements'].apply(lambda x: [elem.strip('\'') for elem in x[1:-1].split(', ')])

In [3]:
import json
with open('mpids_to_lattice_type.json', 'r') as f:
    mpids_to_lattice_type = json.load(f) 
with open('lattice_type_dict.json', 'r') as f:
    lattice_type_dict = json.load(f)

In [69]:
print(mp_stat.groupby('lattice_type').count()['mpid'])
print("Total",mp_stat.groupby('lattice_type').count()['mpid'].sum())

lattice_type
cubic           17133
hexagonal        8681
monoclinic      28081
orthorhombic    24115
tetragonal      14031
triclinic       14843
trigonal        10651
Name: mpid, dtype: int64
Total 117535


In [13]:
mp_stat.drop(columns=['mpid','elements']).groupby('lattice_type').mean()

,a,b,c,avg,grid_x,grid_y,grid_z,grid,n_atoms
lattice_type,,,,,,,,,
cubic,6.708461,7.090869,9.381009,7.726780,103.181463,109.021304,143.750306,112.953723,24.361058
hexagonal,6.678405,7.057662,9.359250,7.698439,102.684944,108.462620,143.402834,112.371767,23.791153
monoclinic,6.670595,7.051213,9.353765,7.691857,102.627186,108.437093,143.354154,112.372271,24.019408
orthorhombic,6.646257,7.021584,9.392689,7.686843,102.299150,108.002322,143.903960,112.098894,23.741240
tetragonal,6.653519,7.046792,9.384191,7.694834,102.332264,108.332407,143.736013,112.217148,23.991305
triclinic,6.635191,7.036416,9.341519,7.671042,102.107930,108.220171,143.185609,112.167329,23.909385
trigonal,6.684336,7.062708,9.284478,7.677174,102.773824,108.568961,142.358088,112.221075,23.717397


In [14]:
mp_stat.drop(columns=['mpid','elements']).groupby('lattice_type').max()

,a,b,c,avg,grid_x,grid_y,grid_z,grid,n_atoms
lattice_type,,,,,,,,,
cubic,31.549540,31.549540,91.100357,32.429992,480,480,1372,480.0,144
hexagonal,34.830905,50.182057,98.709741,34.966023,540,756,1500,448.0,136
monoclinic,65.402600,42.854979,101.386228,35.942256,972,640,1512,500.0,144
orthorhombic,74.229734,74.882108,90.449206,41.504814,1120,1120,1372,540.0,136
tetragonal,38.788312,32.616749,100.462372,35.597565,576,500,1500,480.0,154
triclinic,38.477274,25.276576,88.154727,31.953611,576,384,1344,360.0,144
trigonal,31.527578,31.527578,88.135585,31.947357,480,480,1344,480.0,144


In [15]:
mp_stat.drop(columns=['mpid','elements']).groupby('lattice_type').min()

,a,b,c,avg,grid_x,grid_y,grid_z,grid,n_atoms
lattice_type,,,,,,,,,
cubic,1.611270,1.282899,1.544743,2.083065,24,20,24,31.448897,1
hexagonal,2.280203,2.408562,2.486967,2.521312,36,36,36,40.000000,1
monoclinic,2.174460,2.174520,2.131537,2.174520,32,32,32,32.000000,1
orthorhombic,1.520366,1.776060,1.776060,1.776060,24,28,28,28.000000,1
tetragonal,1.711288,1.711288,2.340089,2.375663,28,28,36,35.277789,1
triclinic,2.052674,2.386838,0.984187,2.070576,32,36,16,29.472252,1
trigonal,1.631531,1.631531,2.459570,2.459376,24,24,36,36.000000,1


In [80]:
lattice_atom_info = {}
for keys in lattice_type_dict.keys():
    lattice_type_dict[keys] = []
    for elemlist in mp_stat[mp_stat['lattice_type'] == keys]['elements']:
        lattice_type_dict[keys].extend(elemlist)
    lattice_type_dict[keys] = list(set(lattice_type_dict[keys]))

In [87]:
from rdkit import Chem

pt = Chem.GetPeriodicTable()
lattice_atom_info = {}

for keys in lattice_type_dict.keys():
    print(keys,len(lattice_type_dict[keys]))
    lattice_atom_info[keys] = [
    {
        'name': a,
        'atom_num': pt.GetAtomicNumber(a),
        'radius': round(pt.GetRcovalent(a) / 0.529177, 5)  # convert to Bohr
    } for a in lattice_type_dict[keys]
    ]
    with open(f'mp_splits/mp_{keys}_atom_info.json', 'w') as f:
        json.dump(lattice_atom_info[keys], f, indent=2)

triclinic 85
monoclinic 85
orthorhombic 87
tetragonal 87
trigonal 86
hexagonal 85
cubic 87
unknown 0
